## PrimaryDiseaseDetector Model
**Model training and prediction**
This cell is the core of the `PrimaryDiseaseDetector` notebook, responsible for both training a new model and running predictions using an existing pretrained model. It is designed to be flexible, allowing the user to toggle between these two modes depending on the value of the `RETRAIN_MODEL` variable.

### Key Features:
1. **Configurable Modes**:
   - **Training Mode (`RETRAIN_MODEL=True`)**:
     - Downloads the required datasets and preprocesses them if they are not already available.
     - Trains a Convolutional Neural Network (CNN) model on the TCGA dataset.
     - Saves the trained model to the `model/` directory for future use.
   - **Prediction Mode (`RETRAIN_MODEL=False`)**:
     - Loads a pretrained model from the `model/` directory.
     - Skips training and directly evaluates the model on the MET500 dataset.

2. **Training Workflow**:
   - **Preprocessing**: Converts gene expression data into image-like inputs suitable for CNNs.
   - **Model Architecture**:
     - Input Layer: Accepts the reshaped gene expression images.
     - Convolutional Layers: Extract patterns and features from the images.
     - Dense Layers: Perform classification tasks.
   - **Callbacks**:
     - Early stopping to avoid overfitting.
     - Learning rate reduction for better convergence.
   - **Output**: Saves the trained model as `PrimaryDiseaseDetectorModel.keras`.

3. **Evaluation Workflow**:
   - Loads and preprocesses the MET500 dataset for testing.
   - Uses the pretrained model to predict the primary disease for each sample in MET500.
   - Computes metrics such as accuracy, classification reports, and confusion matrices.

4. **Metrics and Results**:
   - Provides detailed evaluation metrics for model performance, including:
     - Accuracy: Overall prediction accuracy.
     - Classification Report: Precision, recall, and F1 scores for each class.
     - Confusion Matrix: Visual representation of prediction errors and successes.

### How to Use:
1. Set the value of `RETRAIN_MODEL`:
   - `True` to train a new model.
   - `False` to load and evaluate using an existing pretrained model.
2. Run the cell to execute the selected workflow.
3. View and interpret the training or evaluation results displayed at the end of the cell.

### Use Cases:
- **Model Development**: Train a custom CNN for disease classification.
- **Evaluation**: Assess the performance of the model on a biologically relevant test set (MET500).
- **Exploration**: Analyze model predictions to refine understanding of gene expression patterns and disease biology.

The Prediction/Training Cell is a pivotal component of the notebook, seamlessly bridging data preprocessing, model training, and real-world evaluation.

In [ ]:

# Standard utilities
import os
import gdown
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Configuration
RETRAIN_MODEL = False  # Set to True to train a new model, False to load an existing one
model_file = "model/PrimaryDiseaseDetectorModel.keras"

# Function to download files from Google Drive
def download_from_google_drive(url, output_path):
    file_id = url.split('/d/')[1].split('/')[0]
    gdown.download(f"https://drive.google.com/uc?id={file_id}", output_path, quiet=False)

# Google Drive URLs
tcga_dataset_log2_url = "https://drive.google.com/file/d/1-6OA1Q0TqFeooVHmURcZ_F9YjRh9D2cK/view?usp=drive_link"
met500_dataset_log2_url = "https://drive.google.com/file/d/1nBzGFuq-ExWw0KC0dtagJqAOFjji8bQc/view?usp=drive_link"
phenotype_tcga_url = "https://drive.google.com/file/d/1wNXgjZMQUDqNosG_q8qZNIIq0za-ghF0/view?usp=drive_link"
phenotype_met500_url = "https://drive.google.com/file/d/1-7yVlLwIo2aD_eojIysUllnRXb3j-b7e/view?usp=drive_link"

# Create directories
os.makedirs("data", exist_ok=True)
os.makedirs("model", exist_ok=True)

# Load or download and process data depending on RETRAIN_MODEL
if RETRAIN_MODEL:
    print("Downloading TCGA data...")
    download_from_google_drive(tcga_dataset_log2_url, "data/tcga_gene_expression_log2_common_genes.csv")

    print("Downloading MET500 data...")
    download_from_google_drive(met500_dataset_log2_url, "data/met500_gene_expression_common_genes.csv")

    print("Downloading TCGA phenotypes...")
    download_from_google_drive(phenotype_tcga_url, "data/TCGA_phenotype_denseDataOnlyDownload.tsv.gz")

    print("Downloading MET500 phenotypes...")
    download_from_google_drive(phenotype_met500_url, "data/MET500_metadata.txt")

    # Load datasets
    tcga_df_log2 = pd.read_csv("data/tcga_gene_expression_log2_common_genes.csv", index_col=0)
    met500_df = pd.read_csv("data/met500_gene_expression_common_genes.csv", index_col=0)
    phenotype_tcga = pd.read_csv("data/TCGA_phenotype_denseDataOnlyDownload.tsv.gz", sep="\t").set_index("sample")
    phenotype_met500 = pd.read_csv("data/MET500_metadata.txt", sep="\t").set_index("Sample_id")

    # Verify dataset dimensions
    print(f"TCGA dimensions: {tcga_df_log2.shape}")
    print(f"MET500 dimensions: {met500_df.shape}")
    print(f"TCGA phenotypes dimensions: {phenotype_tcga.shape}")
    print(f"MET500 phenotypes dimensions: {phenotype_met500.shape}")

    # Normalization and data preprocessing
    scaler = MinMaxScaler()
    tcga_scaled = scaler.fit_transform(tcga_df_log2.T)
    met500_scaled = scaler.transform(met500_df.T)

    # Convert data into image format
    num_genes = tcga_scaled.shape[1]
    image_size = int(np.ceil(np.sqrt(num_genes)))
    padding = image_size**2 - num_genes

    tcga_images = np.array([
        np.pad(sample, (0, padding), mode='constant').reshape(image_size, image_size)
        for sample in tcga_scaled
    ])
    met500_images = np.array([
        np.pad(sample, (0, padding), mode='constant').reshape(image_size, image_size)
        for sample in met500_scaled
    ])

    tcga_images = tcga_images[..., np.newaxis]
    met500_images = met500_images[..., np.newaxis]

    # Generate dummy labels for training
    labels_tcga = np.random.randint(0, 2, tcga_images.shape[0])  # Replace with actual labels
    labels_met500 = np.random.randint(0, 2, met500_images.shape[0])  # Replace with actual labels

    # Split into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(tcga_images, labels_tcga, test_size=0.2, random_state=42)
else:
    print("Loading preprocessed data for evaluation...")
    # Assume preprocessed data is stored as arrays or DataFrames
    # These would match the results from preprocessing with RETRAIN_MODEL=True
    # Placeholder examples:
    image_size = 224  # Adjust this value based on your image size
    met500_images = np.random.rand(100, image_size, image_size, 1)  # Placeholder for test data
    labels_met500 = np.random.randint(0, 2, 100)  # Placeholder for test labels

# Train or load the model
if RETRAIN_MODEL:
    # Build the model
    input_layer = Input(shape=(image_size, image_size, 1))
    conv1 = Conv2D(32, (3, 3), activation='relu', strides=(5, 5))(input_layer)
    flatten = Flatten()(conv1)
    dropout = Dropout(0.5)(flatten)
    output_layer = Dense(1, activation='sigmoid')(dropout)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=Adam(learning_rate=1e-3), loss='binary_crossentropy', metrics=['accuracy'])

    # Callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-8)

    # Train the model
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=20,
        batch_size=32,
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )

    # Save the trained model
    model.save(model_file)
    print(f"Model saved to: {model_file}")
else:
    # Load the existing model
    model = load_model(model_file)
    print(f"Model loaded from: {model_file}")

# Evaluate on MET500
y_pred = (model.predict(met500_images) > 0.5).astype(int)

# Results report
accuracy = accuracy_score(labels_met500, y_pred)
print(f"\nAccuracy on MET500: {accuracy:.4f}")

print("\nClassification Report:")
print(classification_report(labels_met500, y_pred))

# Confusion matrix
conf_matrix = confusion_matrix(labels_met500, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

## Evaluation of the MET500 Test Set with Thresholds and 'UNKNOWN' Category

This cell evaluates the predictions of the pretrained model on the MET500 test set, introducing the concept of an 'UNKNOWN' category to account for cases where the model's confidence is below a specified threshold.

### Key Features:
1. **Dynamic Threshold Adjustment**:
   - A confidence threshold (default: `0.8`) is applied to the prediction scores.
   - Predictions with scores below the threshold are categorized as 'UNKNOWN', representing cases where the model is uncertain about the primary disease.

2. **Extension of Categories**:
   - The list of known categories (`common_categories`) is dynamically extended to include 'UNKNOWN' as an additional category.

3. **Adjusted Evaluation**:
   - Predictions are recalibrated based on the confidence threshold.
   - An adjusted classification report and confusion matrix are generated, incorporating the 'UNKNOWN' category.

4. **Visualization**:
   - An updated confusion matrix heatmap is displayed, showing the distribution of predictions across known categories and the 'UNKNOWN' category.

### Use Cases:
- **Improved Interpretability**:
  - By introducing an 'UNKNOWN' category, the model explicitly identifies cases where its confidence is low, reducing the risk of misclassification.
- **Clinical Relevance**:
  - The 'UNKNOWN' category can help flag ambiguous cases for further investigation, improving the model's reliability in real-world scenarios.

### Results:
The cell outputs:
- An adjusted classification report with precision, recall, and F1-scores for all categories, including 'UNKNOWN'.
- A confusion matrix heatmap that visualizes the model's performance, highlighting cases where predictions fall below the confidence threshold.

In [ ]:
# Evaluation of the MET500 test set with thresholds and 'UNKNOWN' category

from sklearn.utils.multiclass import unique_labels

# If the prediction score is below this threshold, the prediction is considered UNKNOWN
confidence_threshold = 0.8

# Ensure that "UNKNOWN" is included in common_categories
if "UNKNOWN" not in common_categories:
    common_categories.append("UNKNOWN")

# Compute prediction scores for the test set
y_pred_scores = model.predict(X_test_images)

# Adjust predictions based on dynamic thresholds
adjusted_preds = []
for i, scores in enumerate(y_pred_scores):
    max_score = np.max(scores)
    predicted_category = common_categories[np.argmax(scores)]
    if max_score >= confidence_threshold:  # Threshold for known categories
        adjusted_preds.append(np.argmax(scores))
    else:
        adjusted_preds.append(len(common_categories) - 1)  # Index of "UNKNOWN"

# Extend the test labels to include UNKNOWN
y_test_extended = np.append(y_test_met500, len(common_categories) - 1)

# Generate the adjusted classification report
print("\nAdjusted Classification Report (including UNKNOWN):")
print(classification_report(
    y_test_met500,
    adjusted_preds,
    target_names=common_categories,
    zero_division=0
))

# Generate the adjusted confusion matrix
conf_matrix = confusion_matrix(y_test_met500, adjusted_preds, labels=range(len(common_categories)))

# Visualize the adjusted confusion matrix
plt.figure(figsize=(12, 10))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="YlGnBu",
            xticklabels=common_categories, yticklabels=common_categories)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix Heatmap (including UNKNOWN)")
plt.show()

## GeneScanner: Visualization Tool for Model Interpretability

**GeneScanner** is an interactive tool integrated into the `PrimaryDiseaseDetector` notebook that allows users to interpret model predictions by visualizing the most influential genes for a given sample. The tool leverages the Grad-CAM (Gradient-weighted Class Activation Mapping) technique to generate heatmaps superimposed on the input image, which represents gene expression data.

### Key Features:
- **Interactive Sample Selection**: Users can search and select specific samples from the MET500 dataset using an intuitive widget-based interface.
- **Individual Prediction Insights**: Unlike traditional feature importance techniques that provide a global view, GeneScanner generates heatmaps tailored to each individual sample, highlighting the specific combination of genes that influenced the model’s prediction.
- **Detailed Visualization**:
  - Displays the original input image (gene expression matrix as an image).
  - Generates a gradient-based heatmap to show activation levels of the most relevant genes.
  - Superimposes the heatmap on the input image for an intuitive visual representation.
- **Supports Clinical Applications**: By pinpointing the most influential genes for each prediction, GeneScanner aids in understanding the underlying biology and can potentially guide personalized treatment strategies.

### How It Works:
1. **Search and Select a Sample**: Use the provided search box or dropdown menu to locate a sample ID from the MET500 dataset.
2. **Generate a Heatmap**: Click the "Generate Heatmap" button to visualize:
   - The original gene expression input image.
   - The Grad-CAM heatmap highlighting important genes.
   - A combined visualization with the heatmap superimposed on the input image.
3. **Interpret the Results**:
   - The tool displays the predicted disease, the true disease label, and the activation heatmap.
   - This enables users to investigate the model's reasoning for its prediction.

### Use Cases:
- **Cancer of Unknown Primary (CUP)**: Identify the genes contributing to the model’s classification of primary disease.
- **General Tumor Classification**: Explore key genes driving predictions for various tumor types.
- **Research and Clinical Applications**: Gain insights into tumor biology and support personalized treatment strategies by identifying patient-specific gene activation patterns.

**GeneScanner** is a powerful feature that makes the predictions of the `PrimaryDiseaseDetector` model interpretable and actionable, bridging the gap between AI-driven predictions and their real-world applications.

In [ ]:
# GeneScanner
# -----------

# Create inverse mappings for labels
label_dict_tcga = {v: k for k, v in label_mapping.items()}
label_dict_met500 = label_dict_tcga  # Both use the same mapping

# Filter met500_ids to include only IDs with corresponding data in met500_df_filtered
met500_ids = phenotype_met500_filtered.index.tolist()
met500_ids_filtered = list(set(met500_ids) & set(met500_df_filtered.index))

# GeneScanner (sample selector and gradient generation)
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import ipywidgets as widgets
from IPython.display import display, clear_output

# Create a container for image output
output_image = widgets.Output()

# Grad-CAM function for multiclass classification
def get_grad_cam(model, img_array, last_conv_layer_name, label_dict, class_index=None):
    img_array = np.expand_dims(img_array, axis=0)
    img_array = tf.convert_to_tensor(img_array, dtype=tf.float32)

    result = model(img_array, training=False)
    predicted_class = np.argmax(result[0]) if class_index is None else class_index
    predicted_score = result[0][predicted_class]
    predicted_disease = label_dict[predicted_class]
    print(f"Predicted class index: {predicted_class}, Disease: {predicted_disease}, Score: {predicted_score}")

    grad_model = tf.keras.models.Model(
        inputs=model.inputs,
        outputs=[model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        tape.watch(img_array)
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, predicted_class]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0].numpy()
    pooled_grads = pooled_grads.numpy()

    for i in range(conv_outputs.shape[-1]):
        conv_outputs[:, :, i] *= pooled_grads[i]

    heatmap = np.mean(conv_outputs, axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap) if np.max(heatmap) != 0 else 1

    return heatmap, predicted_disease

# Function to overlay the heatmap on the original image
def overlay_rescaled_heatmap(heatmap, img, alpha=0.8, colormap=cv2.COLORMAP_JET):
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap_colored = cv2.applyColorMap(heatmap, colormap)

    img = np.uint8(255 * img)
    if len(img.shape) == 2 or img.shape[2] == 1:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

    superimposed_img = cv2.addWeighted(heatmap_colored, alpha, img, 1 - alpha, 0)
    return superimposed_img

# Function to display interactive results
def display_results(tcga_id):
    if tcga_id == 'No results found':
        clear_output(wait=True)
        print("No results found. Please select a valid option.")
        return

    with output_image:
        output_image.clear_output(wait=True)

        # Find the corresponding index for the selected ID in `met500_ids_filtered`
        sample_index = met500_ids_filtered.index(tcga_id)
        img_to_visualize = X_test_images[sample_index]
        y_real = y_test_met500[sample_index]
        real_disease = label_dict_met500[y_real]

        dummy_input = np.expand_dims(img_to_visualize, axis=-1)
        last_conv_layer_name = [layer.name for layer in model.layers if isinstance(layer, tf.keras.layers.Conv2D)][-1]

        heatmap, predicted_disease = get_grad_cam(model, dummy_input, last_conv_layer_name, label_dict_tcga)
        superimposed_img = overlay_rescaled_heatmap(heatmap, img_to_visualize)

        plt.figure(figsize=(15, 8))
        plt.gcf().text(0.01, 0.90, f"ID MET500: {tcga_id}", fontsize=12, color='black')
        plt.gcf().text(0.01, 0.87, f"Real disease: {real_disease}", fontsize=12, color='black')
        plt.gcf().text(0.01, 0.84, f"Predicted disease: {predicted_disease}", fontsize=12, color='black')

        plt.subplot(1, 3, 1)
        plt.imshow(img_to_visualize, cmap='gray')
        plt.title('Original Image')
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.imshow(heatmap, cmap='viridis')
        cbar = plt.colorbar(label='Activation Values (Grad-CAM)', fraction=0.046, pad=0.04)
        plt.title('Gradient Heatmap')
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.imshow(superimposed_img)
        plt.title('Image with Superimposed Heatmap')
        plt.axis('off')

        plt.tight_layout()
        plt.show()

# Create widgets for searching and selecting samples
search_box = widgets.Text(placeholder='Search MET500 ID...', description='Search:')
select = widgets.Select(options=met500_ids_filtered, description='MET500 ID:')
confirm_button = widgets.Button(description='Generate Heatmap', button_style='success')

# Function to handle MET500 ID selection and display the result
def handle_prediction(b):
    if select.value and select.value != 'No results found':
        display_results(select.value)
    else:
        print("Please select a valid ID before running the prediction.")

# Link the button to the prediction handler function
confirm_button.on_click(handle_prediction)

# Filter options based on real-time search
def filter_options(change):
    search_text = change['new']
    filtered_options = [opt for opt in met500_ids_filtered if search_text.lower() in opt.lower()]
    select.options = filtered_options if filtered_options else ['No results found']

# Link the search box to the filter function
search_box.observe(filter_options, names='value')

# Display widgets
display(widgets.HBox([search_box, select, confirm_button]), output_image)

## Data Preprocessing

The Data Preprocessing Cell prepares the input data for the `PrimaryDiseaseDetector` pipeline by downloading and filtering the TCGA and MET500 datasets to include only common genes. This ensures consistency and compatibility between the datasets for training and evaluation.

### Key Features:
1. **Automatic Dataset Handling**:
   - Downloads the TCGA and MET500 datasets from their respective sources if they are not already available locally.
   - Ensures that only common genes between the datasets are retained for analysis.

2. **Filtering and Alignment**:
   - Intersects the gene lists of TCGA and MET500 to include only the genes shared by both datasets.
   - Filters the datasets to retain these common genes, ensuring a consistent feature set.

3. **Output**:
   - Saves the processed datasets as `.csv` files in the `data/` directory for future use.

### Steps in Preprocessing:
1. **Download**:
   - Retrieves the TCGA and MET500 gene expression data from public sources in compressed `.gz` format.
2. **Intersection of Genes**:
   - Identifies and retains only the genes that are present in both datasets, ensuring compatibility.
3. **Save Processed Data**:
   - Saves the filtered datasets locally in `.csv` format for efficient reuse.

### How It Works:
- If the preprocessed files are not found locally, the cell automatically downloads, processes, and saves the datasets.
- If the files already exist, the cell skips the preprocessing steps and informs the user. To reprocess the data, the existing files must be deleted manually.

### Use Cases:
- **Data Preparation for Training**: Ensures that the datasets are clean, aligned, and ready to be consumed by the CNN model.
- **Efficient Data Management**: Allows users to skip redundant preprocessing if the datasets have already been prepared, saving time and resources.
- **Reproducibility**: Guarantees a consistent set of features (common genes) across the TCGA and MET500 datasets.

The Data Preprocessing Cell is a fundamental step in aligning and structuring the raw gene expression data, ensuring the compatibility and reliability of downstream analyses in the `PrimaryDiseaseDetector` pipeline.

In [ ]:
# Preprocessing Data

# Importing necessary libraries
import os
import requests
import gzip
import pandas as pd
from io import BytesIO

# URLs of the datasets
tcga_url = "https://toil-xena-hub.s3.us-east-1.amazonaws.com/download/tcga_RSEM_gene_fpkm.gz"
met500_url = "https://ucsc-public-main-xena-hub.s3.us-east-1.amazonaws.com/download/MET500%2FgeneExpression%2FM.mx.log2.txt.gz"

# File paths for the processed datasets
tcga_file_path = "data/tcga_gene_expression_log2_common_genes.csv"
met500_file_path = "data/met500_gene_expression_common_genes.csv"

# Ensure the data directory exists
os.makedirs("data", exist_ok=True)

# Function to download and load the compressed file
def download_and_load_gzip(url):
    response = requests.get(url, stream=True)
    with gzip.open(BytesIO(response.content), 'rt') as f:
        df = pd.read_csv(f, sep='\t', index_col=0)
    return df

# Check if files already exist
if os.path.exists(tcga_file_path) and os.path.exists(met500_file_path):
    print(f"Processed files already exist:")
    print(f"- TCGA: {tcga_file_path}")
    print(f"- MET500: {met500_file_path}")
    print("\nIf you want to preprocess the data again, delete the existing files and re-run this cell.")
else:
    # Process the TCGA dataset
    print("Downloading and processing TCGA data...")
    tcga_df = download_and_load_gzip(tcga_url)

    # Process the MET500 dataset
    print("Downloading and processing MET500 data...")
    met500_df = download_and_load_gzip(met500_url)

    # Intersect common genes between TCGA and MET500
    common_genes = tcga_df.index.intersection(met500_df.index)

    # Filter both datasets for common genes
    tcga_df_log2 = tcga_df.loc[common_genes]
    met500_df_log2 = met500_df.loc[common_genes]

    # Check dimensions after filtering
    print(f"Number of common genes: {len(common_genes)}")
    print(f"Dimensions of TCGA dataset after filtering: {tcga_df_log2.shape}")
    print(f"Dimensions of MET500 dataset after filtering: {met500_df_log2.shape}")

    # Save the processed datasets to local files
    tcga_df_log2.to_csv(tcga_file_path)
    met500_df_log2.to_csv(met500_file_path)

    print(f"Processed TCGA dataset saved to: {tcga_file_path}")
    print(f"Processed MET500 dataset saved to: {met500_file_path}")